In [1]:
import ee
import time
import geemap
import ipywidgets as widgets

In [2]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=DPyte9_MQAAKTfDb0xXFEKheObfFIx9rvgh1ILbuCfc&tc=SCOgPJSuYniJ5Y_m2dg5K-MV1dGNGqmxdCMotH0hApw&cc=CmWwbK6WI8-g3aGKXjpi8kd59eCX68OU91JoLrdfEoA

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkCFZ7D9QInfEny2p6Cbfj1N1AAPlj4aG40dwMCLW2_a_C4KwCGmFg

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
## function to mask clouds
def maskClouds(image):
    # Select the QA60 band
    QA60 = image.select(['QA60'])

    # Bits 10 and 11 are cloud and cirrus bits, set all others to 0
    cloud_mask = QA60.bitwiseAnd(1 << 10).eq(0).And(QA60.bitwiseAnd(1 << 11).eq(0))

    # Update the image mask to exclude cloudy and shadowed pixels
    return image.updateMask(cloud_mask).divide(10000)

def addBands(image):
    return image.addBands(image.normalizedDifference(['B8','B4']).rename('NDVI'))\
      .addBands(image.normalizedDifference(['B3','B8']).rename('NDWI'))\
      .addBands(image.normalizedDifference(['B8','B12']).rename('NDWI2'))\
      .addBands(image.normalizedDifference(['B11','B8']).rename('NDBI'))\
      .addBands(image.expression('((B4+B11)-(B8+B2))/((B4+B11)+(B8+B2))',{'B2':image.select('B2'),'B8':image.select('B8'),'B4':image.select('B4'),'B11':image.select('B11')}).rename('NDBSI'))\
      .addBands(image.expression('(B4 + 0.3)/(B3 + B11)',{'B3':image.select('B3'),'B4':image.select('B4'),'B11':image.select('B11')}).rename('BAEI'))\
      .addBands(image.expression('(2.5*(B8-B4))/(1+B8+(6*B4)-(7.5*B2))',{'B8':image.select('B8'),'B4':image.select('B4'),'B2':image.select('B2')}).rename('EVI'))

def addBands2(image):
    return image.addBands(image.expression('NDBI-NDVI',{'NDBI':image.select('NDBI'),'NDVI':image.select('NDVI')}).rename('BUI'))


In [5]:
## this function computes different characteristics for each polygon contained in the dataset
## after the hurricane, adds the label 'status' to classify later other objects
def medianReducer(image):
    result = image.reduceRegion(
        reducer = ee.Reducer.median(), scale = 10, maxPixels = 1e9, bestEffort = True
    )
    return result
def getCharacteristics(feature):
    coords = feature.geometry()
    image2 = ee.Image(ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                    .filterDate('2023-10-25', '2023-11-03')
                    .select(['B4','B3','B2','B8','B11','B12','QA60'])
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 90))
                    .map(maskClouds)
                    .median()).clip(coords)
    image2 = addBands(image2)
    image2 = addBands2(image2)

    clipped = image2

    b4 = medianReducer(clipped.select(['B4']))
    b3 = medianReducer(clipped.select(['B3']))
    b2 = medianReducer(clipped.select(['B2']))
    b8 = medianReducer(clipped.select(['B8']))
    b11 = medianReducer(clipped.select(['B11']))
    b12 = medianReducer(clipped.select(['B12']))
    ndvi = medianReducer(clipped.select(['NDVI']))
    ndwi = medianReducer(clipped.select(['NDWI']))
    ndwi2 = medianReducer(clipped.select(['NDWI2']))
    ndbi = medianReducer(clipped.select(['NDBI']))
    ndbsi = medianReducer(clipped.select(['NDBSI']))
    baei = medianReducer(clipped.select(['BAEI']))
    evi = medianReducer(clipped.select(['EVI']))
    bui = medianReducer(clipped.select(['BUI']))

    return ee.Feature(coords,ee.Dictionary({'B4': b4.get('B4'),'B3': b3.get('B3'),'B2': b2.get('B2'),'B8': b8.get('B8'),'B11': b11.get('B11'),'B12': b12.get('B12'),
        'NDVI': ndvi.get('NDVI'),'NDWI': ndwi.get('NDWI'),'NDWI2': ndwi2.get('NDWI2'),'NDBI': ndbi.get('NDBI'),'NDBSI': ndbsi.get('NDBSI'),'BAEI': baei.get('BAEI'),
        'EVI': evi.get('EVI'),'BUI': bui.get('BUI')}))



## function to only export polygons
def getPolygons(feature):
    return ee.Feature(feature).set('geometry_type', ee.Feature(feature).geometry().type())



In [6]:
## previously load ESA's public data for acapulco in EE
## Load the shapefile as a FeatureCollection
asset1 = 'users/eeranyartrodrigo/datathon/coyuca_benitez_manzanas'
data = ee.FeatureCollection(asset1)

In [7]:
data.first()

In [8]:
data_characterized = data.map(getCharacteristics)

polygons = data_characterized.map(getPolygons).filter(ee.Filter.equals('geometry_type', 'Polygon'))
polygons = ee.FeatureCollection(polygons)

params = {
  'collection': polygons,
  'description': 'coyuca_benitez_characterized_data_python',
  'fileFormat': 'SHP',
  'folder': 'datathon_training_data_acapulco'
}

# Export the table to Google Drive
task = ee.batch.Export.table.toDrive(**params)
task.start()

# Monitor the task status
while task.status()['state'] in ['READY', 'RUNNING']:
    print('Task is', task.status()['state'])
    time.sleep(120)  # Optional: Add a delay to reduce API calls frequency

# Check if the export was successful
if task.status()['state'] == 'COMPLETED':
    print('Export completed successfully. Check your Google Drive folder.')
else:
    print('Export failed: ', task.status()['state'])

Task is READY
Task is RUNNING
Task is RUNNING
Export completed successfully. Check your Google Drive folder.
